In [ ]:
import os
from dotenv import load_dotenv
from crewai import Agent, LLM, Task, Crew, Process

# Load environment variables
load_dotenv()

# Option D: Azure OpenAI
openai_api_key = os.getenv("AZURE_OPENAI_API_KEY")
openai_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
openai_api_version = os.getenv("AZURE_OPENAI_API_VERSION")
openai_model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

llm = LLM(
    model="azure/gpt-4o-mini",
    api_key=openai_api_key,
    base_url=openai_endpoint,
    api_version=openai_api_version,
    azure=True
)

print("🚀 Environment configured!")
print(f"LLM Model: {llm.model}")

## Reference other tasks

In [13]:
from pydantic import BaseModel

class ResearchFindings(BaseModel):
    """Structured research report output"""
    title: str
    key_findings: list[str]

class AnalysisSummary(BaseModel):
    """Structured summary of research findings"""
    insights: list[str]
    key_takeaways: str


In [14]:
from crewai import Agent, LLM

# AI Agents
research_agent = Agent(
    role="AI Researcher",
    goal="Find and summarize the latest AI advancements",
    backstory="You are an expert AI researcher who stays up to date with the latest innovations.",
    llm=llm,
    verbose=True,
)

analysis_agent = Agent(
    role="AI Analyst",
    goal="Analyze AI research findings and extract key insights",
    backstory="You are a data analyst who extracts valuable insights from research data.",
    llm=llm,
    verbose=True,
)

writer_agent = Agent(
    role="Tech Writer",
    goal="Write a well-structured blog post on AI trends",
    backstory="You are a technology writer skilled at transforming complex AI research into readable content.",
    llm=llm,
    verbose=True,
)


In [15]:
from crewai import Task

# Step 1: Research Task
research_task = Task(
    description="Find and summarize the latest AI advancements",
    expected_output="A structured list of recent AI breakthroughs",
    agent=research_agent,
    output_pydantic=ResearchFindings  # Structured output
)

# Step 2: Analysis Task (References Research Task Output)
analysis_task = Task(
    description="Analyze AI research findings and extract key insights",
    expected_output="A structured summary with key takeaways",
    agent=analysis_agent,
    output_pydantic=AnalysisSummary,
    context=[research_task]  # Receives output from research_task
)

# Step 3: Blog Writing Task (References Both Research and Analysis)
blog_writing_task = Task(
    description="Write a detailed blog post about AI trends",
    expected_output="A well-structured blog post",
    agent=writer_agent,
    context=[research_task, analysis_task]  # Uses both research and analysis outputs
)


In [ ]:
from crewai import Crew

ai_research_crew = Crew(
    agents=[research_agent, analysis_agent, writer_agent],
    tasks=[research_task, analysis_task, blog_writing_task],
    verbose=True
)

# Execute the workflow
result = ai_research_crew.kickoff()

# Print the final blog post
print("\n=== Generated Blog Post ===")
print(result.raw)


In [ ]:
analysis_task.output

In [ ]:
analysis_task.output.pydantic